## Machine Learning


### Apresentação do Dataset

In [1]:
import pandas as pd
import numpy as np
# execute a linha abaixo apenas se não tiver o Sweetviz
# ! pip install sweetviz


#### Link para Download do Dataset

In [2]:
df = pd.read_excel('saidamerged.xlsx', usecols=['Título do Ajuste - conforme relatório SUMÁRIO-001',
       'Descrição do Ajuste', 'Descrição de Impacto', 'Impacto Principal',
       'Observação', 'splitedajuste', 'NOME', 'TIPO', 'Justificativa'])
df.head()

,Título do Ajuste - conforme relatório SUMÁRIO-001,Descrição do Ajuste,Descrição de Impacto,Impacto Principal,Observação,splitedajuste,NOME,TIPO,Justificativa
0,Encargo Liquidação MCP - Original - Principal (*),Inadimplência do MCP (Mercado de Curto Prazo) ...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Principal,NaN,Sem categoria,NaN
1,Encargo Liquidação MCP - Estorno - Principal (*),Estorno da Inadimplência do MCP (Mercado de Cu...,Estorno da inadimplência dos agentes CELESC DI...,-,-,Encargo Liquidação MCP - Estorno - Principal,NaN,Sem categoria,NaN
2,Encargo Liquidação MCP - Original - Juros (*),Lançamento dos Juros sobre a Inadimplência do ...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Juros,NaN,Sem categoria,NaN
3,Encargo Liquidação MCP - Estorno - Juros (*),Estorno dos juros sobre a inadimplência do MCP...,Este ajuste impacta a crédito os agentes que p...,-,-,Encargo Liquidação MCP - Estorno - Juros,NaN,Sem categoria,NaN
4,Encargo Liquidação MCP - Original - Atualizaçã...,Lançamento da atualização monetária sobre a In...,São impactados os Agentes inadimplentes da liq...,-,-,Encargo Liquidação MCP - Original - Atualizaçã...,NaN,Sem categoria,NaN


In [3]:
df.columns

Index(['Título do Ajuste - conforme relatório SUMÁRIO-001',
       'Descrição do Ajuste', 'Descrição de Impacto', 'Impacto Principal',
       'Observação', 'splitedajuste', 'NOME', 'TIPO', 'Justificativa'],
      dtype='object')

In [4]:
# importing sweetviz
import sweetviz as sv
#analyzing the dataset
advert_report = sv.analyze(df)
#display the report
advert_report.show_html('Advertising.html')


Report Advertising.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [11]:
df.TIPO.value_counts()

ENERGIA           663
Sem categoria     571
ESS               325
Não considerar    211
Name: TIPO, dtype: int64

In [6]:
df['Impacto Principal'].unique()

array(['-', 'Todos os impactos decorrentes de uma recontabilização',
       'Impacto no MRE',
       'Nível de contratação (CCEARs) das distribuidoras. ',
       'Nível de contratação (CCEARs) das distribuidoras ',
       'Nivél de contratação (CCEARs) das distribuidoras. ', nan,
       'Pagamento de encargos por segurança energética.',
       'Pagamento de encargos', 'Encargos',
       'Todos os impactos decorrentes de uma recontabilização .',
       'Perdas de rede básica.', 'Contratação por disponibilidade.',
       'Pagamento de encargos.', 'Encargos.',
       'Rateio para pagamento de encargo por restrição de operação quando PLD<INC<=CMO.',
       'Nível de contratação (CCEN) das distribuidoras ',
       'Impacto no RRH',
       'Nivel de contratação (CCEARs) das distribuidoras ',
       'Balanço Energético e MRE', 'Balanço Energético',
       'Nivél de contratação (CCEARs) das distribuidoras ',
       'Pagamento de encargos por segurança energética',
       'Impacto Encargos'], d